In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_small = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [5]:
print(df_small.head())
print('-' * 75)
print(df_small.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [6]:
def add_fractal(df, i, look_back=3):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan

In [7]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['ema30'] = talib.EMA(df['Mid_Close'], timeperiod=30)
df['ema25'] = talib.EMA(df['Mid_Close'], timeperiod=25)
df['ema20'] = talib.EMA(df['Mid_Close'], timeperiod=20)
df['ema15'] = talib.EMA(df['Mid_Close'], timeperiod=15)
df['slowk'], df['slowd'] = talib.STOCH(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=500)
# df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# print(df['fractal'].value_counts())

In [2]:
value_per_pip = 1.0
# max_pips_to_risk = 0.0020
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, spread_cutoff, use_tl, min_pips_to_risk, max_pips_to_risk, adx_cutoff):
    max_pips_to_risk /= 10000
    min_pips_to_risk /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None

    i = 12

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']
        ema200 = df.loc[df.index[i - 1], 'ema200']
        ema175 = df.loc[df.index[i - 1], 'ema175']
        ema150 = df.loc[df.index[i - 1], 'ema150']
        ema125 = df.loc[df.index[i - 1], 'ema125']
        ema100 = df.loc[df.index[i - 1], 'ema100']
        ema75 = df.loc[df.index[i - 1], 'ema75']
        ema10_1, ema5_1 = df.loc[df.index[i - 1], ['ema10', 'ema5']]
        ema10_2, ema5_2 = df.loc[df.index[i - 2], ['ema10', 'ema5']]
        adx = max(df.loc[df.index[i - 6:i], 'adx'])
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        curr_ml2 = df.loc[df.index[i - 2], 'Mid_Low']
        curr_ml1 = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh2 = df.loc[df.index[i - 2], 'Mid_High']
        curr_mh1 = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh1 - curr_ml1)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [0, macd2, macdsignal1]
        trending = adx >= adx_cutoff
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
          df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                              'Ask_Low', 'Ask_Close']]

        if trade is None:
          if macd2 < macdsignal2 and macd1 > macdsignal1 and ema5_1 > ema10_1 and ema5_2 > ema10_2 and max(macd_vals) == 0 and enough_volatility and trending:
              all_buys = True

              if all_buys:
                open_price = float(curr_ask_open)
                pullback = None
                pullback = min(df.loc[df.index[i - 12:i], 'Mid_Low'])

                if open_price - pullback < min_pips_to_risk:
                  pullback = open_price - min_pips_to_risk

                elif open_price - pullback > max_pips_to_risk:
                  pullback = open_price - max_pips_to_risk

                if pullback is not None:
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback - pullback_cushion, 3)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                              'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_buys += 1

                            pips_risked.append(curr_pips_to_risk)

          elif macd2 > macdsignal2 and macd1 < macdsignal1 and ema5_1 < ema10_1 and ema5_2 < ema10_2 and min(macd_vals) == 0 and enough_volatility and trending:
              all_sells = True

              if all_sells:
                open_price = float(curr_bid_open)
                pullback = None
                pullback = max(df.loc[df.index[i - 15:i], 'Mid_High'])

                if pullback - open_price < min_pips_to_risk:
                  pullback = open_price + min_pips_to_risk

                elif pullback - open_price > max_pips_to_risk:
                  pullback = open_price + max_pips_to_risk

                if pullback is not None:
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback + pullback_cushion, 3)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                    'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_sells += 1

                            pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          # if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
          #     trade['stop_loss'] = curr_bid_high - trade['pips_risked']
          if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'buy' and curr_bid_high - trade['open_price'] >= trade['pips_risked']:
              trade['stop_loss'] = trade['open_price']
              reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
              day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
              trade['n_units'] = trade['n_units'] / 2
              

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          # if trade is not None and trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
          #   trade['stop_loss'] = trade['pips_risked'] + curr_ask_low
          if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'sell' and trade['open_price'] - curr_ask_low >= trade['pips_risked']:
              trade['stop_loss'] = trade['open_price']
              reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
              day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
              trade['n_units'] = trade['n_units'] / 2

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [10]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1, 1.5, 2]
# risk_reward_ratio_vals = [2]
spread_cutoffs = [1.5, 2, 2.5]
# spread_cutoffs = [2]
use_tl_vals = [True, False]
min_pips_to_risk_vals = [5, 10]
max_pips_to_risk_vals = [15, 20]
adx_cutoff_vals = [5, 10, 15, 20]
n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(use_tl_vals) * len(min_pips_to_risk_vals) * len(max_pips_to_risk_vals) * len(adx_cutoff_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for spread_val in spread_cutoffs:
    for use_tl in use_tl_vals:
      for min_pips in min_pips_to_risk_vals:
        for max_pips in max_pips_to_risk_vals:
          for adx_cutoff in adx_cutoff_vals:
            all_combos.append((risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, adx_cutoff))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_use_tl_val = None
best_min_pips_val = None
best_max_pips_val = None
best_adx_val = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, adx_cutoff in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, adx_cutoff)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
  if min_item is None or reward > min_item['reward']:
    if min_item is not None:
      best_rewards.remove(min_item)
    best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'use_tl': use_tl, 'min_pips': min_pips, 'max_pips': max_pips, 'adx_cutoff': adx_cutoff})


  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_spread_cutoff = spread_val
    best_use_tl_val = use_tl
    best_min_pips_val = min_pips
    best_max_pips_val = max_pips
    best_adx_val = adx_cutoff

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 648

0
Num buys: 0
Num sells: 0
Num trades: 0
Num wins: 0
Num losses: 0
Win streak: 0
Loss streak: 0
Remaining runs: 647
Best reward so far: 0

0
Num buys: 0
Num sells: 0
Num trades: 0
Num wins: 0
Num losses: 0
Win streak: 0
Loss streak: 0
Remaining runs: 646
Best reward so far: 0

-14.000000000002899
Num buys: 1
Num sells: 1
Num trades: 2
Num wins: 0
Num losses: 2
Win streak: 0
Loss streak: 2
Remaining runs: 645
Best reward so far: 0

-42.0000000000087
Num buys: 1
Num sells: 3
Num trades: 4
Num wins: 0
Num losses: 4
Win streak: 0
Loss streak: 4
Remaining runs: 644
Best reward so far: 0

-10.999999999994348
Num buys: 0
Num sells: 2
Num trades: 2
Num wins: 0
Num losses: 2
Win streak: 0
Loss streak: 2
Remaining runs: 643
Best reward so far: 0

-71.00000000002105
Num buys: 3
Num sells: 3
Num trades: 6
Num wins: 1
Num losses: 5
Win streak: 1
Loss streak: 5
Remaining runs: 642
Best reward so far: 0

-7.499999999993623
Num buys: 0
Num sells: 1
Num trades: 1
Num wins: 0
Num losses: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best TL val: ' + str(best_use_tl_val))
print('Best min pips to risk: ' + str(best_min_pips_val))
print('Best max pips to risk: ' + str(best_max_pips_val))
print('Best adx val: ' + str(best_adx_val))
print('-----------------------')
print('Top results:')
for entry in best_rewards:
    print(entry)

# 2018-2019:
# {'reward': 1539, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 1554, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 575, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': -94, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 553, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 526, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 153, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1345, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 585, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': -99, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 25, 'spread': 2}
# {'reward': 150, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': -94, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 1355, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': -96, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 25, 'spread': 3}
# {'reward': 115, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 102, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': -100, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': -125, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': -92, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': -139, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 3}

# 2019-2020:
# {'reward': 494, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 1306, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1316, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1088, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1345, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 892, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 801, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 562, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 495, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 895, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': -34, 'ratio': 2, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1077, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1323, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': -17, 'ratio': 2, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 741, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 549, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 742, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 797, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 350, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 348, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}

# {'reward': 1788, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1795, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 803, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1211, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1335, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 889, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 1329, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 889, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 728, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 787, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1042, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 1431, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 728, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1320, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1425, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1057, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 1089, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1341, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1222, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1093, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}


# 2020-2021:
# {'reward': 2381, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 15, 'spread': 3}
# {'reward': 2239, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 2513, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 2666, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 10}
# {'reward': 2254, 'ratio': 2, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 10}
# {'reward': 2516, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 10}
# {'reward': 2256, 'ratio': 2, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 2277, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 2254, 'ratio': 2, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 2366, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 15, 'spread': 2}
# {'reward': 2531, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 2552, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 2285, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 2667, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 2238, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 10}
# {'reward': 2280, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 10}
# {'reward': 2552, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 10}
# {'reward': 2369, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 15, 'spread': 10}
# {'reward': 2673, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 2523, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}

# {'reward': 549, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1254, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 717, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 734, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 25, 'spread': 2}
# {'reward': 560, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 517, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 528, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 503, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 729, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 732, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 25, 'spread': 3}
# {'reward': 642, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 710, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 495, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 655, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 779, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1259, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 725, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 772, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 674, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 664, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}


------------ FINAL RESULTS ------------
Best reward: 1126.0000000000127
Best risk/reward ratio: 1.9
Best pullback cushion: 20
Best atr percentage: 0.35
Best last x bars: 2
Best fractal distance: 15
Best bar length: inf
